In [104]:
# General Utilities 
import  random, os, pandas as pd, numpy as np
import matplotlib.pyplot as plt, datetime as dt
import dotenv, os, requests, time

# Environment & Dask Client
plt.rcParams['figure.figsize'] = (10,4)
os.makedirs("OutputData", exist_ok=True)
dotenv.load_dotenv(dotenv.find_dotenv(filename=".env"))
from dask.distributed import Client, LocalCluster

# Dune Client
from dune_client.client import DuneClient
from dune_client.query import QueryBase

# Features 
import talib

# Models
from arch import arch_model
import statsmodels.api as sm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

## 🔧 Configuration

In [ ]:
# Key Constants
TARGET_COIN = "ethereum"
BASE_FIAT   = "usd"
TOP_N       = 10
LOOKBACK_DAYS = 365
START_DATE = (dt.datetime.now() - dt.timedelta(days=LOOKBACK_DAYS)).strftime("%Y-%m-%d")
TODAY= dt.date.today().strftime('%Y-%m-%d')
TIMEZONE = "Europe/Madrid"
SMA_WINDOWS = (10, 20, 50)
EMA_WINDOWS = (10, 20, 50)
TRAIN_SPLIT = 0.90
# --- GARCH config ---
GARCH_SCALE = 100.0
GARCH_REFIT_EVERY = 5
SEQ_LEN = 7
# --- LSTM configuration ---
LSTM_UNITS = 64
LSTM_EPOCHS = 25
LSTM_BATCH = 16
# --- Dune API configuration ---
DUNE_QUERIES = {
    "economic_security": 1933076,
    "daily_dex_volume": 4388,
    "btc_etf_flows": 5795477,
    "eth_etf_flows": 5795645,
    "total_defi_users": 2972,
    "median_gas": 2981260,
}
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
DUNE_CSV_PATH = "OutputData/Dune_Metrics.csv"

# --- FRED API configuration ---
FRED_API_KEY= os.getenv("FRED_API_KEY")
FRED_KNOWN = {
    "VIXCLS":   "vix_equity_vol",            # CBOE VIX (Equity market volatility index)
    "MOVE":     "move_bond_vol",             # ICE BofA MOVE Index (Bond market volatility)
    "OVXCLS":   "ovx_oil_vol",               # CBOE Crude Oil Volatility Index (Oil market volatility)
    "GVZCLS":   "gvz_gold_vol",              # CBOE Gold Volatility Index (Gold market volatility)
    "DTWEXBGS": "usd_trade_weighted_index",  # Trade-Weighted U.S. Dollar Index (Broad Goods)
    "DGS2":     "us_2y_treasury_yield",      # U.S. 2-Year Treasury Yield (constant maturity)
    "DGS10":    "us_10y_treasury_yield",     # U.S. 10-Year Treasury Yield (constant maturity)
}

# --- REPRODUCABILITY ---
np.random.seed(42)


##  Utilities: Metrics & Plotting

In [106]:
# MAE & MASE (predicted vs. realized)
def mae(y_true, y_pred):
    y_true = pd.Series(y_true).astype(float).values
    y_pred = pd.Series(y_pred).astype(float).values
    return float(np.mean(np.abs(y_true - y_pred)))

def mase(y_true, y_pred):
    y_true = pd.Series(y_true).astype(float).values
    y_pred = pd.Series(y_pred).astype(float).values
    naive = np.roll(y_true, 1)[1:]
    err_model = np.abs(y_true[1:] - y_pred[1:]).mean()
    err_naive = np.abs(y_true[1:] - naive).mean()
    return float(err_model / err_naive) if err_naive > 0 else np.nan


##  APIs



In [ ]:
# API Modules
# Expects these globals to be defined by the notebook:
# TIMEZONE, DAYS_BACK, CG_TOP_N, CG_HEADERS,
# DUNE_CSV_PATH, FRED_API_KEY (env)

# --- CoinGecko ---
def CoinGecko_GetUniverse(n, cg_headers=None):
    if cg_headers is None: 
        cg_api_key = os.getenv("COINGECKO_API_KEY")
        cg_headers = {
            "accept": "application/json",
            "x_cg_demo_api_key": cg_api_key
            }
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
    js = requests.get(url, headers=cg_headers).json()
    df = pd.DataFrame(js)
    try:
        return  df.head(n)['id'].values 
    except: 
        return print("Error Getting Coin Id's: ", df.loc['error_message'].values)
    
def CoinGecko_GetPriceAction(coins= CoinGecko_GetPriceAction(TOP_N), start= START_DATE, tz=TIMEZONE, cg_api_key=os.getenv("COINGECKO_API_KEY")):
    cg_headers = {
    "accept": "application/json",
    "x_cg_demo_api_key": cg_api_key
    }
    end   = int(dt.datetime.now(dt.timezone.utc).timestamp()) * 1000
    start = int(pd.to_datetime(START_DATE).timestamp())) * 1000
    count= 0
    for c in coins:
        try:
            url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start}&to={end}"
            js = requests.get(url, headers=cg_headers).json()
            p = pd.DataFrame(js["prices"],        columns=["t", f"prices_{c}"])
            m = pd.DataFrame(js["market_caps"],   columns=["t", f"marketcaps_{c}"])
            v = pd.DataFrame(js["total_volumes"], columns=["t", f"total_volumes_{c}"])
            df = p.merge(m, on="t").merge(v, on="t")
            df.index = pd.DatetimeIndex(df['t'], freq='D', tz=TIMEZONE)
            df = df.set_index("t")
            df.columns = [x.lower() for x in df.columns]
            df.index = df.index.tz_convert(timezone)
            df = df.resample("1D").last().dropna(how="any")
            df.index.name = "date"
            if count ==0: out = df
            else: out = out.join(df, how='inner')
            count= count+1
        except Exception as e:
            print(f"Error for {c}: {e}")
            continue
        time.sleep(5)  # Add delay to avoid rate limits
    return out

# --- Deribit DVOL ---
def Deribit_GetDVOL(currencies, days, timezone, resolution="1D"):
    out = None
    end   = int(dt.datetime.now(dt.timezone.utc).timestamp()) * 1000
    start = int((dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=days)).timestamp()) * 1000
    count=0
    for cur in currencies:
        js = requests.post(
            "https://www.deribit.com/api/v2/",
            json={"method": "public/get_volatility_index_data",
                    "params": {"currency": cur, "resolution": resolution,
                                "end_timestamp": end, "start_timestamp": start}}
        ).json()
        data = js.get("result", {}).get("data", [])
        if not data:
            continue
        d = pd.DataFrame(data, columns=["t","open","high","low","dvol"])
        d["t"] = pd.to_datetime(d["t"], unit="ms", utc=True)
        df = d.set_index("t")[["dvol"]].rename(columns={"dvol": f"dvol_{cur.lower()}"})
        df.index = df.index.tz_convert('Europe/Madrid')
        df = df.resample("1D").last().dropna(how="any")
        df.index.name = "date"
        if count ==0: out = df
        else: out = out.join(df, how='inner')
        count= count+1
    return out

# --- Dune (CSV) ---    
def Dune_FromCSV(path, timezone):
    if not os.path.exists(path):
        return pd.DataFrame()
    df = pd.read_csv(path, index_col=None)
    dt_col = None
    for c in df.columns:
        try:
            pd.to_datetime(df[c], utc=True, errors="raise")
            dt_col = c
            break
        except Exception:
            continue
    if dt_col is None and "date" in df.columns:
        dt_col = "date"
    if dt_col is None:
        return pd.DataFrame()
    df = df.rename(columns={dt_col: "date"})
    df["date"] = pd.to_datetime(df["date"], utc=True, errors="coerce")
    df = df.set_index("date")
    df.index = df.index.tz_convert(timezone)
    df.columns = [c.lower() for c in df.columns]
    df.index.name = "date"
    df = df.resample("1D").last().dropna(how="any")
    return df

# --- Dune ---
def Dune_GetQueries(query_ids, timezone, dune_api_key=None):
    dune = DuneClient(api_key=dune_api_key or os.environ.get("DUNE_API_KEY"),
                       base_url="https://api.dune.com")
    out = None
    for qid in query_ids:
        try:
            q = QueryBase(query_id=qid)
            df = dune.run_query_dataframe(query=q, ping_frequency=2, batch_size=365)
            ok = False
            for col in list(df.columns):
                try:
                    pd.to_datetime(df[col], utc=True, errors="raise")
                    df = df.rename(columns={col: "date"}).set_index("date")
                    ok = True
                    break
                except:
                    continue
            if not ok and not isinstance(df.index, pd.DatetimeIndex):
                continue
            if isinstance(df.index, pd.DatetimeIndex):
                df.index = df.index.tz_convert(timezone)
            df.columns = [c.lower() for c in df.columns]
            df.index.name = "date"
            df = df.resample("1D").last().dropna(how="any")
            out = df if out is None else out.join(df, how="inner")
        except:
            continue
    return out if out is not None else print('Error Fetching Dune Queries')

# --- FRED ---
def Fred_GetSeries(series_ids= FRED_KNOWN, start=START_DATE, timezone=TIMEZONE, fred_api_key=FRED_API_KEY):
    key = fred_api_key or os.getenv("FRED_API_KEY")
    if not key:
        return print("No API Key Available")
    base = "https://api.stlouisfed.org/fred/series/observations"
    df= None
    for sid in series_ids:
        try:
            js = requests.get(base, params={
                    "series_id": sid, "api_key": fred_api_key, "file_type": "json",
                    "observation_start": start
                }).json()
            obs= pd.DataFrame(js['observations'])
            index = pd.DatetimeIndex(obs['date'], freq='infer', tz=timezone)
            obs = obs.set_index(index)['value'].rename(FRED_KNOWN[sid])
            if df is not None: df= pd.merge(left= df, right=obs, left_index=True, right_index=True)
            else: df = obs
        except:
            print("error fetching:", series_ids[sid])
            continue
        time.sleep(2)
    if df is not None:  return df.asfreq('D', method='ffill')
    else: return print('Error Compiling Data')

In [ ]:
def CoinGecko_GetPriceAction(coins= CoinGecko_GetPriceAction(TOP_N), start= START_DATE, tz=TIMEZONE, cg_api_key=os.getenv("COINGECKO_API_KEY")):
    cg_headers = {
    "accept": "application/json",
    "x_cg_demo_api_key": cg_api_key
    }
    end_timestamp   = int(dt.datetime.now().timestamp()) * 1000
    start_timestamp = int(pd.to_datetime(START_DATE).timestamp()) * 1000
    count= 0
    for c in coins:
        try:
            url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start_timestamp}&to={end_timestamp}"
            js = requests.get(url, headers=cg_headers).json()

In [217]:
coins= CoinGecko_GetUniverse(10)
start= START_DATE
tz=TIMEZONE
cg_api_key=os.getenv("COINGECKO_API_KEY")
cg_headers = {
    "accept": "application/json",
    "x_cg_demo_api_key": cg_api_key
    }
end_timestamp   = int(dt.datetime.now().timestamp()) * 1000
start_timestamp = int(pd.to_datetime(START_DATE).timestamp()) * 1000
count= 0
dfs={}
outputbig=None
for c in coins:
    try:
        url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start_timestamp}&to={end_timestamp}"
        dfs[c] = requests.get(url, headers=cg_headers).json()
        outputsmall= None
        for key in dfs[c].keys():
            if key == 'status':
                print(f"Error compiling price data for {c}:\n{dfs[c]['status']['error_message']}")
                continue
            else:
                df= pd.DataFrame(dfs[c][key], columns=['date', key+'_'+c])
                index= index= pd.DatetimeIndex(pd.to_datetime(df['date'], unit='ms'), tz=TIMEZONE)
                df[key+'_'+c].index =  index
                series= df[key+'_'+c]
                if outputsmall is not None: outputsmall= pd.concat([outputsmall, series], axis=1)
                else: outputsmall = series
        if outputbig is not None: outputbig = outputbig.join(outputsmall, how='outer')
        else: outputbig = outputsmall
        time.sleep(2)
    except: 
        time.sleep(10)
        continue
outputbig

Error compiling price data for binancecoin:
You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits.
Error compiling price data for solana:
You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits.
Error compiling price data for usd-coin:
You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits.
Error compiling price data for staked-ether:
You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits.
Error compiling price data for dogecoin:
You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits.
Error compiling price data for tron:
You've exceeded the Rate Limit. Please visit https://

,prices_bitcoin,market_caps_bitcoin,total_volumes_bitcoin,prices_ethereum,market_caps_ethereum,total_volumes_ethereum,prices_tether,market_caps_tether,total_volumes_tether,prices_ripple,market_caps_ripple,total_volumes_ripple
date,,,,,,,,,,,,
2024-09-26 00:00:00+02:00,63151.899594,1.245721e+12,2.608764e+10,2578.566360,3.098750e+11,1.400368e+10,1.000612,1.193308e+11,4.583612e+10,0.583895,3.296539e+10,1.184865e+09
2024-09-27 00:00:00+02:00,65130.768840,1.287151e+12,3.799557e+10,2630.949837,3.168290e+11,1.668616e+10,1.000071,1.193339e+11,6.199968e+10,0.590002,3.333829e+10,1.320296e+09
2024-09-28 00:00:00+02:00,65791.002125,1.299708e+12,3.266492e+10,2698.192821,3.246816e+11,1.646812e+10,1.000440,1.194817e+11,3.959733e+10,0.589115,3.332935e+10,1.401604e+09
2024-09-29 00:00:00+02:00,65934.107094,1.303218e+12,1.534291e+10,2680.218702,3.227619e+11,9.607073e+09,1.000640,1.196467e+11,2.389775e+10,0.615009,3.478693e+10,2.770313e+09
2024-09-30 00:00:00+02:00,65663.689867,1.297826e+12,1.294871e+10,2659.611212,3.201461e+11,9.570811e+09,0.999926,1.195088e+11,3.519751e+10,0.642592,3.634053e+10,2.594473e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-22 00:00:00+02:00,115304.479994,2.297332e+12,1.865911e+10,4452.871130,5.374069e+11,1.591068e+10,1.000400,1.720058e+11,5.412936e+10,2.973350,1.778269e+11,2.935908e+09
2025-09-23 00:00:00+02:00,112696.741017,2.245929e+12,6.915234e+10,4199.951774,5.071486e+11,5.298809e+10,1.000815,1.721574e+11,1.486343e+11,2.851139,1.704006e+11,9.546312e+09
2025-09-24 00:00:00+02:00,112022.165879,2.232693e+12,4.615488e+10,4166.190550,5.032099e+11,2.990268e+10,1.000206,1.727859e+11,9.586917e+10,2.829056,1.691131e+11,5.177734e+09


In [136]:
dfs[c]['status']['error_message']

"You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."

In [133]:
for key in dfs[c]:
    print(key)

status


In [122]:
pd.to_datetime(START_DATE, timezone=TIMEZONE)

TypeError: to_datetime() got an unexpected keyword argument 'timezone'

In [120]:
coins= CoinGecko_GetUniverse(TOP_N)
days=LOOKBACK_DAYS
timezone=TIMEZONE
cg_api_key=None
if cg_api_key is None: 
    cg_api_key = os.getenv("COINGECKO_API_KEY")
cg_headers = {
    "accept": "application/json",
    "x_cg_demo_api_key": cg_api_key
    }
end   = int(dt.datetime.now(tz=TIMEZONE).timestamp()) * 1000
start = int((dt.datetime.now(tz=TIMEZONE) - dt.timedelta(days=days)).timestamp()) * 1000
count= 0
for c in coins:
    try:
        url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start}&to={end}"
        js = requests.get(url, headers=cg_headers).json()
        p = pd.DataFrame(js["prices"],        columns=["t", f"prices_{c}"])
        m = pd.DataFrame(js["market_caps"],   columns=["t", f"marketcaps_{c}"])
        v = pd.DataFrame(js["total_volumes"], columns=["t", f"total_volumes_{c}"])
        df = p.merge(m, on="t").merge(v, on="t")
        df.index = pd.DatetimeIndex(df['t'], freq='D', tz=TIMEZONE)
        df = df.set_index("t")
        df.columns = [x.lower() for x in df.columns]
        df.index = df.index.tz_convert(timezone)
        df = df.resample("1D").last().dropna(how="any")
        df.index.name = "date"
        if count ==0: out = df
        else: out = out.join(df, how='inner')
        count= count+1
    except Exception as e:
        print(f"Error for {c}: {e}")
        continue
    time.sleep(5)  # Add delay to avoid rate limits
# return out

TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'str'

## Techincal Analysis Indicators (ta-lib)

In [108]:
# Technical Analysis Indicators
def TaLib_ComputeIndicators(df, price_prefix="prices_", rsi_period=14,
                          macd_fast=12, macd_slow=26, macd_signal=9,
                          sma_windows=(10,20,50), ema_windows=(10,20,50)):
    out = pd.DataFrame(index=df.index)
    price_cols = [c for c in df.columns if c.startswith(price_prefix)]
    if not price_cols: return out
    coins = [c[len(price_prefix):] for c in price_cols]
    for coin in coins:
        p = pd.to_numeric(df[f"{price_prefix}{coin}"], errors="coerce")
        out[f"rsi{rsi_period}_{coin}"] = talib.RSI(p.values, timeperiod=rsi_period)
        macd, macd_sig, macd_hist = talib.MACD(p.values, fastperiod=macd_fast, slowperiod=macd_slow, signalperiod=macd_signal)
        out[f"macd_{coin}"] = macd; out[f"macd_signal_{coin}"] = macd_sig; out[f"macd_hist_{coin}"] = macd_hist
        for w in sma_windows: out[f"sma{w}_{coin}"] = talib.SMA(p.values, timeperiod=w)
        for w in ema_windows: out[f"ema{w}_{coin}"] = talib.EMA(p.values, timeperiod=w)
    out.index = df.index
    return out


##  GARCH(1,1) & HAR-RV 

In [109]:
# HAR & GARCH
def fit_garch_11(returns, scale=100.0):
    r = returns.dropna().astype(float) * scale
    am = arch_model(r, mean="Zero", vol="GARCH", p=1, q=1, dist="normal")
    return am.fit(disp="off")
def forecast_garch_rolling(returns, train_size, scale=100.0, refit_every=0):
    r = returns.dropna().astype(float); idx=r.index; n=len(r)
    preds, pred_idx, last_refit, res = [], [], -1, None
    for t in range(train_size, n):
        if (res is None) or (refit_every==0) or ((t-last_refit)>=refit_every):
            res = fit_garch_11(r.iloc[:t], scale=scale); last_refit = t
        var_next = res.forecast(horizon=1).variance.iloc[-1,0]
        preds.append(float(np.sqrt(var_next))/scale); pred_idx.append(idx[t])
    return pd.Series(preds, index=pred_idx, name="garch11_vol_pred")
def _har_features(rv):
    rv = rv.astype(float)
    return pd.DataFrame({"RV1": rv.shift(1), "RV5": rv.shift(1).rolling(5).mean(), "RV22": rv.shift(1).rolling(22).mean()})
def fit_har_ols(rv_train):
    X = _har_features(rv_train); y = rv_train
    df = pd.concat([X, y.rename("y")], axis=1).dropna()
    model = sm.OLS(df["y"], sm.add_constant(df[["RV1","RV5","RV22"]])).fit()
    p = model.params.to_dict()
    return {"const": p.get("const",0.0), "RV1": p["RV1"], "RV5": p["RV5"], "RV22": p["RV22"]}
def forecast_har(rv, params, start_idx):
    rv = rv.astype(float); idx = rv.index; n=len(rv); preds=[]
    for t in range(start_idx, n):
        rv1 = rv.iloc[t-1] if t-1>=0 else np.nan
        rv5 = rv.iloc[max(0,t-5):t].mean()
        rv22 = rv.iloc[max(0,t-22):t].mean()
        preds.append(float(np.dot([1.0, rv1, rv5, rv22], [params["const"], params["RV1"], params["RV5"], params["RV22"]])))
    return pd.Series(preds, index=idx[start_idx:], name="har_vol_pred")


##  LSTM 

In [110]:
# LSTM
def make_univariate_sequences(series, seq_len):
    v = series.dropna().astype(float).values
    X, y = [], []
    for i in range(len(v)-seq_len):
        X.append(v[i:i+seq_len]); y.append(v[i+seq_len])
    return np.asarray(X,float).reshape(-1,seq_len,1), np.asarray(y,float)
def build_lstm_model(input_shape, units=64, dropout=0.0):
    m = Sequential(); m.add(LSTM(units, input_shape=input_shape))
    if dropout>0: m.add(Dropout(dropout))
    m.add(Dense(1)); m.compile(optimizer="adam", loss="mse"); return m
def train_lstm(model, X_train, y_train, epochs=25, batch_size=16, validation_split=0.1, patience=5):
    cb=[EarlyStopping(monitor="val_loss", patience=patience, restore_best_weights=True)] if validation_split and patience else []
    return model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_split=validation_split,callbacks=cb,verbose=0,shuffle=False)
def predict_lstm(model, X_seq): return model.predict(X_seq, verbose=0).reshape(-1)


## Assemble Unified DataFrame with Modules

,vix_equity_vol,ovx_oil_vol,gvz_gold_vol,usd_trade_weighted_index,us_2y_treasury_yield,us_10y_treasury_yield
date,,,,,,
2024-09-26 00:00:00+02:00,15.37,39.76,18.82,121.3274,3.6,3.79
2024-09-27 00:00:00+02:00,16.96,39.62,18.59,121.356,3.55,3.75
2024-09-28 00:00:00+02:00,16.96,39.62,18.59,121.356,3.55,3.75
2024-09-29 00:00:00+02:00,16.96,39.62,18.59,121.356,3.55,3.75
2024-09-30 00:00:00+02:00,16.73,39.86,18.08,121.5298,3.66,3.81
...,...,...,...,...,...,...
2025-09-15 00:00:00+02:00,15.69,30.98,17.9,120.0042,3.54,4.05
2025-09-16 00:00:00+02:00,16.36,33.17,18.37,119.586,3.51,4.04
2025-09-17 00:00:00+02:00,15.72,31.05,17.41,119.6063,3.52,4.06


Error for bitcoin: Inferred frequency 86400us from passed values does not conform to passed frequency D
Error for ethereum: Inferred frequency 86400us from passed values does not conform to passed frequency D
Error for tether: Inferred frequency 86400us from passed values does not conform to passed frequency D
Error for ripple: Inferred frequency 86400us from passed values does not conform to passed frequency D
Error for binancecoin: 'prices'
Error for solana: 'prices'
Error for usd-coin: 'prices'
Error for dogecoin: 'prices'
Error for staked-ether: 'prices'
Error for tron: 'prices'


UnboundLocalError: cannot access local variable 'out' where it is not associated with a value

In [ ]:
timezone=TIMEZONE
cg_api_key= os.environ.get("COINGECKO_API_KEY")
cg_headers = {
            "accept": "application/json",
            "x_cg_demo_api_key": cg_api_key
            }
end_timestamp   = int(dt.datetime.now(tz=timezone).timestamp()) * 1000
start_timestamp = int((dt.datetime.now(tz=timezone) - dt.timedelta(days=days)).timestamp()) * 1000
url = f"https://api.coingecko.com/api/v3/coins/{Coins[1]}/market_chart/range?vs_currency=usd&from={start_timestamp}&to={end_timestamp}"
js = requests.get(url, headers=cg_headers).json()

In [ ]:
if cg_api_key is None: 
        inenv= os.getenv("COINGECKO_API_KEY")
        if inenv is None: print("No CoinGecko API key available")
        else: cg_api_key= inenv
    cg_headers = {
            "accept": "application/json",
            "x_cg_demo_api_key": cg_api_key
            }
    end_timestamp   = int(dt.datetime.now(tz=timezone).timestamp()) * 1000
    start_timestamp = int((dt.datetime.now(tz=timezone) - dt.timedelta(days=days)).timestamp()) * 1000
    count= 0
    for c in coins:
        try:
            url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start_timestamp}&to={end_timestamp}"
            js = requests.get(url, headers=cg_headers).json()
            p = pd.DataFrame(js["prices"],        columns=["t", f"prices_{c}"])
            m = pd.DataFrame(js["market_caps"],   columns=["t", f"marketcaps_{c}"])
            v = pd.DataFrame(js["total_volumes"], columns=["t", f"total_volumes_{c}"])
            df = p.merge(m, on="t").merge(v, on="t")
            df.index = pd.DatetimeIndex(df['t'], freq='D', tz=timezone)
            df = df.drop("t")
            df.columns = [x.lower() for x in df.columns]
            df.index = df.index.tz_convert(timezone)
            df = df.resample("1D").last().dropna(how="any")
            df.index.name = "date"
            if count ==0: out = df
            else: out = out.join(df, how='inner')
            count= count+1
        except Exception as e:
            print(f"Error for {c}: {e}")
            continue
        time.sleep(5)  # Add delay to avoid rate limits
    return out

In [ ]:
Unified.dropna(thresh= int(0.1*len(Unified.columns)))